In [1]:
import os

In [2]:
os.getcwd()

'/home/ubuntu/projects/llm-models/llm-models/notebook'

In [3]:
# import models.bidirectional.bidirGPT as bidirGPT

In [4]:
import importlib, os

from models.deepseek import model
importlib.reload(model)

/usr/bin/ld: cannot find -laio: No such file or directory
collect2: error: ld returned 1 exit status
/usr/bin/ld: cannot find -laio: No such file or directory
collect2: error: ld returned 1 exit status
/home/ubuntu/projects/llm-dev/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


<module 'models.deepseek.model' from '/home/ubuntu/projects/llm-models/llm-models/models/deepseek/model.py'>

In [5]:
import importlib, os

from models.deepseek import model
importlib.reload(model)

<module 'models.deepseek.model' from '/home/ubuntu/projects/llm-models/llm-models/models/deepseek/model.py'>

In [6]:
import sys
sys.path.insert(0, "/home/ubuntu/projects/Megatron-LM")
from gpt_builders import gpt_builder

In [7]:
import torch.distributed as dist

os.environ['MASTER_ADDR'] = 'localhost'
os.environ['MASTER_PORT'] = '12356'
dist.init_process_group(backend='nccl', rank=0, world_size=1)

In [8]:

# args = args_from_yaml
# args = SimpleNamespace(**vars(getattr(args, 'language_model')), **vars(args.model_parallel))

In [9]:
# getattr(args, 'scaled_init_method')

In [10]:
from functools import partial
from megatron.training.yaml_arguments import load_yaml, core_transformer_config_from_yaml
from megatron.training import get_args
from types import SimpleNamespace

from megatron.training.initialize import initialize_megatron

# Load YAML and build transformer config once
args_from_yaml = load_yaml("/home/ubuntu/projects/llm-models/llm-models/models/bidirectional/gpt_config.yaml")
# args_dict = vars(args_from_yaml).copy()

transformer_config = core_transformer_config_from_yaml(args_from_yaml, "language_model")

transfomer_key = 'language_model'
# 1. Convert the base YAML object to a dictionary
args_dict = vars(args_from_yaml).copy()

# 2. Safely merge the nested dictionaries
# Using .update() ensures that if a key exists in both, the nested one takes precedence
transfomer_key = 'language_model'
if hasattr(args_from_yaml, transfomer_key):
    args_dict.update(vars(getattr(args_from_yaml, transfomer_key)))

if hasattr(args_from_yaml, 'model_parallel'):
    args_dict.update(vars(args_from_yaml.model_parallel))

# 3. Create the namespace from the merged dictionary
args = SimpleNamespace(**args_dict)

initialize_megatron(
    extra_args_provider=None,
    args_defaults=vars(args),
    ignore_unknown_args=True # Useful if your YAML has extra fields
)

# Provide gpt_builder with that config (config=... skips internal config loading)
def model_provider_with_yaml_config(pre_process=True, post_process=True, vp_stage=None):
    # You must still have get_args() returning the full args (e.g. from same YAML)
    # args = get_args()

        # Initalize and get arguments, timers, and Tensorboard writer.
    extra_args_provider, args_defaults, get_embedding_ranks, get_position_embedding_ranks = None, None, None, None
    store = None
    
    return gpt_builder(args, pre_process, post_process, vp_stage, config=transformer_config)

model = model_provider_with_yaml_config()
# Then in pretrain:
# pretrain(..., partial(model_provider, model_provider_with_yaml_config), ...)

warning using experimental yaml arguments feature, argparse arguments will be ignored
test
True
using world size: 1, data-parallel size: 1, context-parallel size: 1, hierarchical context-parallel sizes: None, tensor-model-parallel size: 1, pipeline-model-parallel size: 1
Number of virtual stages per pipeline stage: None
using torch.float32 for parameters ...
------------------------ arguments ------------------------
  _cpu_offloading_context ......................... None
  account_for_embedding_in_pipeline_split ......... False
  account_for_loss_in_pipeline_split .............. False
  accumulate_allreduce_grads_in_fp32 .............. False
  activation_func ................................. swiglu
  activation_func_clamp_value ..................... None
  activation_func_fp8_input_store ................. False
  adam_beta1 ...................................... 0.9
  adam_beta2 ...................................... 0.999
  adam_eps ........................................ 1e-08
  

/home/ubuntu/projects/Megatron-LM/megatron/core/transformer/transformer_config.py:1495: UserWarning: apply_rope_fusion for multi-latent attention only supports training. It is experimental and may change in future versions.
  warnings.warn(
/home/ubuntu/projects/Megatron-LM/megatron/core/transformer/transformer_config.py:1668: UserWarning: full scope is deprecated. Use empty cuda_graph_scope to capture the whole layer.
  warnings.warn(
/home/ubuntu/projects/Megatron-LM/megatron/training/utils.py:410: UserWarning: Overriding default arguments for use_legacy_models:False with use_legacy_models:False
  warnings.warn(message)
/home/ubuntu/projects/Megatron-LM/megatron/training/utils.py:410: UserWarning: Overriding default arguments for check_for_nan_in_loss_and_grad:True with check_for_nan_in_loss_and_grad:True
  warnings.warn(message)
/home/ubuntu/projects/Megatron-LM/megatron/training/utils.py:410: UserWarning: Overriding default arguments for add_position_embedding:False with add_positi

  one_logger_project .............................. megatron-lm
  one_logger_run_name ............................. None
  onnx_safe ....................................... None
  openai_gelu ..................................... False
  optimizer ....................................... adam
  optimizer_cpu_offload ........................... False
  optimizer_offload_fraction ...................... 1.0
  original_max_position_embeddings ................ 4096
  outer_dp_sharding_strategy ...................... no_shard
  output_bert_embeddings .......................... False
  output_layer_init_method ........................ kaiming_uniform
  overlap_cpu_optimizer_d2h_h2d ................... False
  overlap_grad_reduce ............................. False
  overlap_moe_expert_parallel_comm ................ False
  overlap_p2p_comm ................................ False
  overlap_p2p_comm_warmup_flush ................... False
  overlap_param_gather ............................ False
 

make: python3-config: No such file or directory
make: python3-config: No such file or directory
/home/ubuntu/projects/Megatron-LM/megatron/training/utils.py:410: UserWarning: Constraints for invoking optimized fused softmax kernel are not met. We default back to unfused kernel invocations.
  warnings.warn(message)
/home/ubuntu/projects/llm-dev/lib/python3.12/site-packages/torch/distributed/distributed_c10d.py:4876: UserWarning: barrier(): using the device under current context. You can specify `device_id` in `init_process_group` to mute this warning.
  warnings.warn(  # warn only once
[rank0]:[W203 08:40:22.340820603 ProcessGroupNCCL.cpp:5068] Guessing device ID based on global rank. This can cause a hang if rank to GPU mapping is heterogeneous. You can specify device_id in init_process_group()


>>> done with compiling and loading fused kernels. Compilation time: 0.345 seconds
building GPT model ...


In [11]:
import torch

print("Allocated:", torch.cuda.memory_allocated() / 1024**2, "MB")
print("Reserved:", torch.cuda.memory_reserved() / 1024**2, "MB")

Allocated: 140.8388671875 MB
Reserved: 158.0 MB


In [12]:
import inspect

In [13]:
model

GPTModel(
  (embedding): LanguageModelEmbedding(
    (word_embeddings): VocabParallelEmbedding()
    (embedding_dropout): Dropout(p=0.0, inplace=False)
  )
  (decoder): TransformerBlock(
    (layers): ModuleList(
      (0-3): 4 x BidirTransformerLayer(
        (input_layernorm): RMSNorm()
        (self_attention): BidirMLASelfAttention(
          (core_attention): FlexDotProductAttention()
          (linear_proj): TERowParallelLinear(in_features=1024, out_features=128, bias=False, TP=1)
          (rotary_pos_emb): YarnRotaryEmbedding()
          (linear_q_down_proj): TELinear()
          (linear_q_up_proj): TEColumnParallelLinear(in_features=512, out_features=1536, bias=False, TP=1)
          (linear_kv_down_proj): TELinear()
          (linear_kv_up_proj): TEColumnParallelLinear(in_features=512, out_features=2048, bias=False, TP=1)
          (q_layernorm): IdentityOp()
          (kv_layernorm): IdentityOp()
          (core_attention_backward): FlexDotProductAttention()
          (bidir

In [14]:

embedding_shape = model.embedding.word_embeddings.weight.shape

with torch.no_grad():
    
    model.embedding.word_embeddings.weight.copy_(torch.zeros(embedding_shape))
    model.embedding.word_embeddings.weight[0].copy_(torch.rand(embedding_shape[1]))

In [15]:
model.embedding.word_embeddings.weight

Parameter containing:
tensor([[0.0290, 0.4019, 0.2598,  ..., 0.6084, 0.0928, 0.0719],
        [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
        ...,
        [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000]],
       device='cuda:0', requires_grad=True)

In [16]:
class ActivationMonitor:
    def __init__(self, model):
        self.hooks = []
        self.activations = dict()
        
        for name, module in model.named_modules():
            # Filter for layers you care about (e.g., Norm, Linear, Attention)
            # if isinstance(module, (torch.nn.Linear, torch.nn.LayerNorm)):
            hook = module.register_forward_hook(self.make_forward_hook(name))
            self.hooks.append(hook)

    def make_forward_hook(self, name):
        def hook(module, act_input, act_output):
            # grad_output[0] is the gradient of the loss w.r.t. the output of this layer
        
            self.activations[name] = act_output
                    
        return hook
        
    def remove(self):
        for h in self.hooks:
            h.remove()

In [17]:
monitor = ActivationMonitor(model)

In [18]:
# monitor.remove()

In [19]:
model.embedding.word_embeddings.weight

Parameter containing:
tensor([[0.0290, 0.4019, 0.2598,  ..., 0.6084, 0.0928, 0.0719],
        [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
        ...,
        [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000]],
       device='cuda:0', requires_grad=True)

In [20]:
import torch

In [21]:
input_ids = torch.ones(2,128, dtype=torch.int32)
input_ids = input_ids.to('cuda')

In [22]:
input_ids[0, 20] = 0
input_ids[0, 30] = 0
input_ids[0, 50] = 0

input_ids[1, 15] = 0
input_ids[1, 25] = 0
input_ids[1, 45] = 0

In [23]:
input_ids

tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1]], device='cuda:0', dtype=torch.int32)

In [24]:
output = model(input_ids, None, None)

torch.Size([128, 2, 128])
core_attn_out_backward_to_backward
tensor([   0.0000,    0.0000,    0.0000,    0.0000,    0.0000,    0.0000,
           0.0000,    0.0000,    0.0000,    0.0000,    0.0000,    0.0000,
           0.0000,    0.0000,    0.0000,    0.0000,    0.0000,    0.0000,
           0.0000,    0.0000,    0.0000,    0.0000,    0.0000,    0.0000,
           0.0000,    0.0000,    0.0000,    0.0000,    0.0000,    0.0000,
           0.0000,   71.8074,   68.5434,   65.5633,   62.8315,   60.3182,
          57.9983,   55.8502,   53.8556,   51.9985,   50.2652,   97.2874,
          94.2472,   91.3912,   88.7033,   86.1689,   83.7753,   81.5111,
          79.3661,   77.3310, 1050.9423,   73.5588,   71.8074,   70.1375,
          68.5434,   67.0202,   65.5633,   64.1683,   62.8315,   61.5492,
          60.3182,   88.7033,   86.9974,   85.3560,   83.7753,   82.2521,
          80.7833,   79.3661,   77.9977,   76.6757,   75.3978,   74.1618,
          72.9656,   71.8074,   70.6854,   69.5979,

In [26]:
monitor.activations.keys()

dict_keys(['embedding.word_embeddings', 'embedding.embedding_dropout', 'embedding', 'decoder.layers.0.input_layernorm', 'decoder.layers.0.self_attention.linear_q_down_proj', 'decoder.layers.0.self_attention.linear_kv_down_proj', 'decoder.layers.0.self_attention.q_layernorm', 'decoder.layers.0.self_attention.kv_layernorm', 'decoder.layers.0.self_attention.linear_q_up_proj', 'decoder.layers.0.self_attention.linear_kv_up_proj', 'decoder.layers.0.self_attention.core_attention_backward', 'decoder.layers.0.self_attention.bidir_attention_backward', 'decoder.layers.0.self_attention.bidir_attention_forward', 'decoder.layers.0.self_attention.linear_proj', 'decoder.layers.0.self_attention', 'decoder.layers.0.pre_cross_attn_layernorm', 'decoder.layers.0.cross_attention', 'decoder.layers.0.cross_attn_bda', 'decoder.layers.0.pre_mlp_layernorm', 'decoder.layers.0.mlp.linear_fc1', 'decoder.layers.0.mlp.linear_fc2', 'decoder.layers.0.mlp', 'decoder.layers.0', 'decoder.layers.1.input_layernorm', 'decode

In [76]:
monitor.activations['decoder.layers.0.self_attention.core_attention_backward'][0].shape

torch.Size([2, 1024])

In [77]:
monitor.activations['decoder.layers.0.self_attention.core_attention_forward_to_backward'][0].shape

torch.Size([2, 1024])

In [81]:
monitor.activations['decoder.layers.0.self_attention.core_attention_forward'][3].shape

torch.Size([2, 1024])

In [82]:
len(monitor.activations['decoder.layers.0.self_attention.core_attention_forward'])

128

In [67]:
monitor.activations['decoder.layers.0.self_attention.linear_q_up_proj'][0][

tensor([[[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]],

        [[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]],

        [[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]],

        ...,

        [[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]],

        [[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]],

        [[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]]], device='cuda:0',
       grad_fn=<_LinearBackward>)

In [65]:
torch.sum(torch.abs(monitor.activations['decoder.layers.0.mlp'][0][:, 0, :]), dim=-1)

tensor([0.0365, 0.0310, 0.0266, 0.0231, 0.0203, 0.0179, 0.0159, 0.0142, 0.0128,
        0.0116, 0.4773, 0.0365, 0.0336, 0.0310, 0.0287, 0.0266, 0.0248, 0.0231,
        0.0216, 0.0203, 0.4773, 0.0839, 0.0761, 0.0694, 0.0636, 0.0585, 0.0540,
        0.0501, 0.0466, 0.0434, 0.1057, 0.0998, 0.0943, 0.0893, 0.0847, 0.0804,
        0.0764, 0.0728, 0.0693, 0.0662, 0.4773, 0.0604, 0.0578, 0.0553, 0.0530,
        0.0509, 0.0489, 0.0469, 0.0452, 0.0434, 0.0623, 0.0600, 0.0579, 0.0559,
        0.0540, 0.0522, 0.0504, 0.0488, 0.0472, 0.0457, 0.0443, 0.0429, 0.0416,
        0.0404, 0.0392, 0.0381, 0.0370, 0.0359, 0.0349, 0.0339, 0.0330, 0.0321,
        0.0313, 0.0305, 0.0297, 0.0289, 0.0282, 0.0275, 0.0268, 0.0262, 0.0255,
        0.0249, 0.0243, 0.0238, 0.0232, 0.0227, 0.0222, 0.0217, 0.0212, 0.0208,
        0.0203, 0.0199, 0.0194, 0.0190, 0.0187, 0.0183, 0.0179, 0.0175, 0.0172,
        0.0169, 0.0165, 0.0162, 0.0159, 0.0156, 0.0153, 0.0150, 0.0147, 0.0145,
        0.0142, 0.0140, 0.0137, 0.0135, 

In [38]:
monitor.activations['decoder.layers.2.self_attention.kv_layernorm'].shape

torch.Size([128, 2, 512])

In [45]:
torch.sum(monitor.activations['decoder.layers.2.self_attention.kv_layernorm'][:, 0, :], dim=-1)

tensor([ 0.0756,  0.1465,  0.2092,  0.2620,  0.3079,  0.3436,  0.3740,  0.3990,
         0.4162,  0.4311,  1.4825,  1.4497,  1.4253,  1.4049,  1.3814,  1.3611,
         1.3433,  1.3297,  1.3216,  1.3129, -1.7523,  2.0350,  1.9856,  1.9344,
         1.8830,  1.8321,  1.7864,  1.7571,  1.7371,  1.7264, -1.7528,  1.1020,
         1.1481,  1.1880,  1.2228,  1.2453,  1.2774,  1.2982,  1.3299,  1.3458,
         1.5958,  2.3061,  2.2492,  2.2012,  2.1502,  2.1090,  2.0653,  2.0375,
         2.0174,  2.0075, -1.7450,  2.5770,  2.5405,  2.5012,  2.4693,  2.4347,
         2.4002,  2.3598,  2.3247,  2.2960,  2.2709,  2.0933,  2.0839,  2.0754,
         2.0612,  2.0494,  2.0353,  2.0126,  1.9946,  1.9783,  1.9650,  2.2816,
         2.2746,  2.2670,  2.2602,  2.2480,  2.2308,  2.2065,  2.1903,  2.1726,
         2.1616,  2.1571,  2.1436,  2.1345,  2.1295,  2.1181,  2.1087,  2.0955,
         2.0807,  2.0716,  2.0572,  2.0454,  2.0361,  2.0337,  2.0222,  2.0202,
         2.0097,  2.0089,  2.0041,  1.99

In [46]:
torch.sum(torch.abs(monitor.activations['decoder.layers.1.self_attention.kv_layernorm'][:, 0, :]), dim=-1)

tensor([ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
         0.0000,  0.0000, 16.1385, 15.3944, 14.7198, 14.1040, 13.5382, 13.0171,
        12.5349, 12.0872, 11.6713, 11.2813, 92.0106, 21.3193, 20.6569, 20.0322,
        19.4414, 18.8866, 18.3655, 17.8717, 17.4036, 16.9596, 92.0110, 32.7712,
        31.6329, 30.5748, 29.5888, 28.6667, 27.8030, 26.9936, 26.2313, 25.5144,
        31.7010, 41.8403, 40.8284, 39.8634, 38.9433, 38.0580, 37.2097, 36.3909,
        35.6114, 34.8556, 92.0095, 33.4430, 32.7712, 32.1287, 31.5075, 30.9155,
        30.3403, 29.7849, 29.2521, 28.7325, 28.2362, 34.5880, 34.0012, 33.4327,
        32.8825, 32.3474, 31.8314, 31.3324, 30.8436, 30.3704, 29.9157, 29.4669,
        29.0358, 28.6134, 28.2062, 27.8118, 27.4259, 27.0537, 26.6866, 26.3309,
        25.9854, 25.6492, 25.3203, 25.0018, 24.6911, 24.3870, 24.0924, 23.7982,
        23.5159, 23.2389, 22.9677, 22.7018, 22.4441, 22.1925, 21.9466, 21.7041,
        21.4673, 21.2359, 21.0092, 20.78

In [88]:
act.shape

torch.Size([128, 2, 1024])

In [20]:
act = torch.concat([torch.unsqueeze(x, dim=0) for x in monitor.activations['decoder.layers.0.self_attention.core_attention_backward']], dim=0)
torch.sum(torch.abs(act[:, 0, :]), dim=-1)

tensor([0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 7.6915, 7.0506, 6.5082, 6.0433, 5.6404, 5.2879,
        4.9769, 4.7004, 4.4530, 4.2303, 8.0578, 7.6915, 7.3571, 7.0506, 6.7685,
        6.5082, 6.2672, 6.0433, 5.8349, 5.7221, 5.4585, 5.2879, 5.1277, 4.9769,
        4.8347, 4.7004, 4.5733, 4.4530, 4.3388, 4.2303, 6.1907, 6.0433, 5.9028,
        5.7686, 5.6404, 5.5178, 5.4004, 5.2879, 5.1800, 5.0764, 4.9769, 4.8812,
        4.7891, 4.7004, 4.6149, 4.5325, 4.4530, 4.3762, 4.3020, 4.2303, 4.1610,
        4.0939, 4.0289, 3.9659, 3.9049, 3.8458, 3.7884, 3.7326, 3.6786, 3.6260,
        3.5749, 3.5253, 3.4770, 3.4300, 3.3843, 3.3397, 3.2964, 3.2541, 3.2129,
        3.1728, 3.1336, 3.0954, 3.0581, 3.0217, 2.9861, 2.9514, 2.9175, 2.8843,
        2.8519, 2.8202, 2.7892, 2.7589, 2.7292, 2.7002, 2.6718, 2.6440, 2.6167,
        2.5900, 2.5638, 2.5382, 2.5131, 

In [23]:
act = torch.concat([torch.unsqueeze(x, dim=0) for x in monitor.activations['decoder.layers.0.self_attention.bidir_attention_forward']], dim=0)
torch.sum(torch.abs(act[:, 0, :]), dim=-1)

tensor([7.6915, 7.0506, 6.5082, 6.0433, 5.6404, 5.2879, 4.9769, 4.7004, 4.4530,
        4.2303, 8.2497, 7.6915, 7.3571, 7.0506, 6.7685, 6.5082, 6.2672, 6.0433,
        5.8349, 5.6404, 5.5682, 5.2879, 5.1277, 4.9769, 4.8347, 4.7004, 4.5733,
        4.4530, 4.3388, 4.2303, 6.1907, 6.0433, 5.9028, 5.7686, 5.6404, 5.5178,
        5.4004, 5.2879, 5.1800, 5.0764, 5.0683, 4.8812, 4.7891, 4.7004, 4.6149,
        4.5325, 4.4530, 4.3762, 4.3020, 4.2303, 4.1610, 4.0939, 4.0289, 3.9659,
        3.9049, 3.8458, 3.7884, 3.7326, 3.6786, 3.6260, 3.5749, 3.5253, 3.4770,
        3.4300, 3.3843, 3.3397, 3.2964, 3.2541, 3.2129, 3.1728, 3.1336, 3.0954,
        3.0581, 3.0217, 2.9861, 2.9514, 2.9175, 2.8843, 2.8519, 2.8202, 2.7892,
        2.7589, 2.7292, 2.7002, 2.6718, 2.6440, 2.6167, 2.5900, 2.5638, 2.5382,
        2.5131, 2.4884, 2.4643, 2.4406, 2.4173, 2.3945, 2.3722, 2.3502, 2.3286,
        2.3075, 2.2867, 2.2663, 2.2462, 2.2265, 2.2071, 2.1881, 2.1694, 2.1510,
        2.1329, 2.1152, 2.0977, 2.0805, 

In [22]:
act = torch.concat([torch.unsqueeze(x, dim=0) for x in monitor.activations['decoder.layers.0.self_attention.bidir_attention_backward']], dim=0)
torch.sum(torch.abs(act[:, 0, :]), dim=-1)

tensor([7.6915, 7.0506, 6.5082, 6.0433, 5.6404, 5.2879, 4.9769, 4.7004, 4.4530,
        4.2303, 8.2497, 7.6915, 7.3571, 7.0506, 6.7685, 6.5082, 6.2672, 6.0433,
        5.8349, 5.6404, 5.5682, 5.2879, 5.1277, 4.9769, 4.8347, 4.7004, 4.5733,
        4.4530, 4.3388, 4.2303, 6.1907, 6.0433, 5.9028, 5.7686, 5.6404, 5.5178,
        5.4004, 5.2879, 5.1800, 5.0764, 5.0683, 4.8812, 4.7891, 4.7004, 4.6149,
        4.5325, 4.4530, 4.3762, 4.3020, 4.2303, 4.1610, 4.0939, 4.0289, 3.9659,
        3.9049, 3.8458, 3.7884, 3.7326, 3.6786, 3.6260, 3.5749, 3.5253, 3.4770,
        3.4300, 3.3843, 3.3397, 3.2964, 3.2541, 3.2129, 3.1728, 3.1336, 3.0954,
        3.0581, 3.0217, 2.9861, 2.9514, 2.9175, 2.8843, 2.8519, 2.8202, 2.7892,
        2.7589, 2.7292, 2.7002, 2.6718, 2.6440, 2.6167, 2.5900, 2.5638, 2.5382,
        2.5131, 2.4884, 2.4643, 2.4406, 2.4173, 2.3945, 2.3722, 2.3502, 2.3286,
        2.3075, 2.2867, 2.2663, 2.2462, 2.2265, 2.2071, 2.1881, 2.1694, 2.1510,
        2.1329, 2.1152, 2.0977, 2.0805, 

In [ ]:
from megatron.core import parallel_state

parallel_state.model_parallel_is_initialized()

In [ ]:
from megatron.core.models.gpt import GPTModel
from megatron.core.transformer.spec_utils import import_module

import megatron.core.models.gpt as gpt
import importlib
importlib.reload(gpt)

In [ ]:
pre_process = True
post_process = True
mtp_block_spec = None
vp_stage = None

model = gpt.GPTModel(
    config=config,
    transformer_layer_spec=transformer_layer_spec,
    vocab_size=args.padded_vocab_size,
    max_sequence_length=args.max_position_embeddings,
    pre_process=pre_process,
    post_process=post_process,
    fp16_lm_cross_entropy=args.fp16_lm_cross_entropy,
    parallel_output=True,
    share_embeddings_and_output_weights=not args.untie_embeddings_and_output_weights,
    position_embedding_type=args.position_embedding_type,
    rotary_percent=args.rotary_percent,
    rotary_base=args.rotary_base,
    rope_scaling=args.use_rope_scaling,
    mtp_block_spec=mtp_block_spec,
    vp_stage=vp_stage,
)

In [ ]:
from megatron.training import get_args

In [ ]:
from functools import partial
from megatron.training.yaml_arguments import load_yaml, core_transformer_config_from_yaml

# Load YAML and build transformer config once
args_from_yaml = load_yaml("/home/ubuntu/projects/llm-models/llm-models/models/bidirectional/gpt_config.yaml")
transformer_config = core_transformer_config_from_yaml(args_from_yaml, "language_model")


In [ ]:
from functools import partial
from megatron.training.yaml_arguments import load_yaml, core_transformer_config_from_yaml

# Load YAML and build transformer config once
args_from_yaml = load_yaml("/home/ubuntu/projects/llm-models/llm-models/models/bidirectional/gpt_config.yaml")
transformer_config = core_transformer_config_from_yaml(args_from_yaml, "language_model")

# Provide gpt_builder with that config (config=... skips internal config loading)
def model_provider_with_yaml_config(pre_process=True, post_process=True, vp_stage=None):
    # You must still have get_args() returning the full args (e.g. from same YAML)
    args = get_args()
    return gpt_builder(args, pre_process, post_process, vp_stage, config=transformer_config)

# Then in pretrain:
pretrain(..., partial(model_provider, model_provider_with_yaml_config), ...)